In [ ]:
import os 
import pandas as pd 
import numpy as np
import collections
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
# set working directory (Change for youe)
os.chdir('C:/Users/ander/Google Drive/Columbia/Fall 2019/Capstone/Dotin-Columbia-Castone-Team-Alpha-')

# initilize path
mouse_flat_path = 'Data/Clean Data/mouse_flat_v3.csv'

# read file 
data = pd.read_csv(mouse_flat_path)
data.head()

,Index,user_id,action,cord_x,cord_y,distance,radio,time_since,id,created_at,updated_at,timeElapsed,window_x,window_y,system
0,0,365,s,0,0.0,0.000000,NaN,0.000,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
1,1,365,m,601,167.0,623.770791,NaN,0.220,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
2,2,365,m,602,166.0,1.414214,NaN,0.304,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
3,3,365,c,602,166.0,0.000000,NaN,0.432,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc
4,4,365,m,602,166.0,0.000000,NaN,0.518,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860,pc


In [4]:
# filter users with less than 196 radio count, which is the number of questions in the data 
subset_radio = data.loc[:,['user_id', 'radio']]
subset_radio = subset_radio.dropna(subset = ['radio'])
user_completion = subset_radio.groupby('user_id').count() >= 196
user_completion = user_completion[user_completion['radio'] == True].reset_index()
user_id_who_completed_survey = pd.DataFrame(user_completion['user_id'])
completed_survey_data = data[data.user_id.isin(user_id_who_completed_survey.user_id)].reset_index()
completed_survey_data = completed_survey_data.drop(columns = ['Index','index'])

In [7]:
coordinates =  completed_survey_data[['user_id','cord_x', 'cord_y','action']].to_numpy()

In [13]:
def parse_distance(coord_data):
    """
    Calculate the incremental coordinate for each mouse movement
    :param coord_data:
    :return: A seperate numpy array for the x, y, user id list.
    """
    user_id = np.unique(coord_data[:, 0])[1:]
    total_x = []
    total_y = []
    user_id_list = []
    action_list = []

    for user in user_id:
        index = coord_data[:, 0] == user
        new_index = coord_data[index]

        id_updated = new_index[1:, 0]
        action_updated =new_index[1:,3]

        distance_x = new_index[1:, 1] - new_index[:len(new_index) - 1, 1]
        distance_y = new_index[1:, 2] - new_index[:len(new_index) - 1, 2]

        total_x = np.append([total_x], [distance_x])
        total_y = np.append([total_y], [distance_y])
        
        user_id_list = np.append([user_id_list], [id_updated])
        action_list = np.append([action_list], [action_updated])

    return total_x, total_y, user_id_list, action_list


def parse_directions(id_list, x_coord, y_coord, action):
    """
    Converts coordinate changes into Cardinal direction.

    :param id_list: Unique list of user ids
    :param x_coord: List of the changes in the x coordinate
    :param y_coord: List of the changes in the y coordinates
    :return: A data frame that contains the user id list, x directional changes, and y directional changes,
    calculated cardinal direction
    """

    directions = collections.deque()  # optimized for append operations
    
    for value in range(0, len(id_list)):

        x = x_coord[value]
        y = y_coord[value]
        
        action_value = action[value]
        
        if action_value == 'm':
            if x == 0 and y == 0:
                directions.append('No Movement')
            elif x > 0 and y == 0:
                directions.append('East')
            elif x < 0 and y == 0:
                directions.append('West')
            elif x == 0 and y > 0:
                directions.append('North')
            elif x == 0 and y < 0:
                directions.append('South')
            elif x > 0 and y > 0:
                directions.append('North East')
            elif x > 0 and y < 0:
                directions.append('North West')
            elif x < 0 and y < 0:
                directions.append('South West')
            elif x < 0 and y > 0:
                directions.append('South East')
            else:
                directions.append('TBD')  
        else: 
            directions.append(action_value)

    return pd.DataFrame({'User Id': id_list,
                         'Distance X': x_coord,
                         'Distance Y': y_coord,
                         'Direction': directions})

In [10]:
# parse distance
coord_x, coord_y, coord_user, action_type = parse_distance(coordinates)

In [12]:
set(action_type)

{'c', 'm', 'np', 's'}

In [16]:
# parse direction
direction_data = parse_directions(coord_user, coord_x, coord_y, action_type)
#direction_data.to_csv(f'Models/Q1_Mouse Activity/Data/direction_data.csv')

In [18]:
direction_data

,User Id,Distance X,Distance Y,Direction
0,386,0,0,s
1,386,0,0,s
2,386,0,0,s
3,386,545,286,North East
4,386,0,0,No Movement
...,...,...,...,...
4569341,1837,-5,2,South East
4569342,1837,-6,0,West
4569343,1837,-1,0,West
4569344,1837,0,0,c


In [31]:
flatten_data = pd.DataFrame(columns=['Movement'],index= user_id_who_completed_survey.user_id)